In [32]:
import pandas as pd
import numpy as np
import sys
from scipy.sparse import *
from scipy import *
import pickle
import requests

In [203]:
plays_train = pd.read_csv("data/training_data.csv")
plays_test = pd.read_csv("data/test_data.csv")
plays_all = pd.read_csv("data/ir2017.csv")

True

In [139]:
user_dict_train = {}
groups = plays_train.groupby('user')
for i, group in groups:
    user_id = group['user'].iloc[0]
    track_ids = set(group['track'])
    user_dict_train[user_id] = track_ids 

In [140]:
user_dict_test = {}
groups = plays_test.groupby('user')
for i, group in groups:
    user_id = group['user'].iloc[0]
    track_ids = set(group['track'])
    user_dict_test[user_id] = track_ids 

In [183]:
userplays_train = user_dict_train['41019bc73383f4f196dbd6ee17cc70ef']

In [184]:
userplays_test = user_dict_test['41019bc73383f4f196dbd6ee17cc70ef']

In [185]:
url_track = 'https://api.spotify.com/v1/tracks/'
url_album = 'https://api.spotify.com/v1/albums/'

headers = {
    'Accept': 'application/json',
    #'Authorization': 'BQDtquTwN3JFhD_7COzfIMrT4LsHA6P34Ps2mM4taKV_g_O'
}


In [186]:
# fetch tracks from same album for all the user track
result = list()
albums = list()
s = requests.Session()
tracks = [ i.replace('spotify:track:', '') for i in userplays_train]
### limited to prevent too many requests at once
tracks = tracks[:100]

import time

for track in tracks:
    response = s.get(url_track + track , headers=headers)
    #time.sleep(0.1)
    if response.status_code == 200:
        json = response.json()
        #print(json['album']['id'])
        albums.append(json['album']['id'])
        #print(json['album']['id'])
    else:
        print(response)

for album in albums:
    response = s.get(url_album + album , headers=headers)
    #time.sleep(0.1)
    if response.status_code == 200:
        json = response.json()
        #result.add(json['tracks']['items']['id'])
        album_tracks = [i['id'] for i in json['tracks']['items']]
        for track in album_tracks:
            if track not in plays_train:
                result.append(track)
    else:
        print(response.status_code)

In [194]:
from collections import Counter
cnt = Counter()
for track in result:
    cnt[track] += 1

In [207]:
sortedlist = list(cnt.most_common()[:200])
userplays_result1 = ['spotify:track:'+i[0] for i in sortedlist]
alltracks = plays_all['track'].unique()
userplays_result = [i for i in userplays_result1 if i in alltracks]
len(userplays_result)

37

In [206]:
len(userplays_result1)

200

In [208]:
def get_precision(result_query, result_given):
    if len(result_query) == 0:
        return 0
    else:
        return len(set(result_given).intersection(result_query))/len(result_query)
def get_recall(result_query, result_given):
    if len(result_given) == 0:
        return 0
    else:
        return len(set(result_given).intersection(result_query))/len(result_given)


precision = get_precision(userplays_result, userplays_test)
recall = get_recall(userplays_result, userplays_test)
    
#precision = get_precision([1, 2], [1, 2, 3, 4])
#recall = get_recall([1, 2], [1, 2, 3, 4])
    
print(precision)
print(recall)

0.1891891891891892
0.4666666666666667
